In [3]:
# parameters to run demo, change these as needed

frate = 2.0 # choose either 2.0 or 5.0
fname_in = 'test_video.avi'

import sys
if ((frate != 2.0) and (frate != 5.0)):
    print 'Current analysis must be done with 2.0 or 5.0 fps'
    exit()

In [4]:
# this reads in the avi

import os
import imageio
import sys
# you might to run this if you don't have ffmpeg.exe
#imageio.plugins.ffmpeg.download()

home = os.getcwd()
fname_full = home + '/' + fname_in
vid = imageio.get_reader(fname_full, 'ffmpeg')
vid_info = vid.get_meta_data()
s_hz = vid_info['fps']
s_length = vid_info['duration']

min_duration = 1.0/frate
if (s_length < min_duration):
    print 'the video is too short'
    print 'the minimum duration is ',min_duration
    print 'and the length of this video is ',duration
    exit()


In [5]:
# This takes the frames of the avi and converts them into the .hkl files
# The script is based on process_kitti

import numpy as np
from scipy.misc import imread, imresize
import hickle as hkl
import pandas as pd

desired_im_sz = (128, 160)

# resize and crop image
def process_im(im, desired_sz):
    target_ds = float(desired_sz[0])/im.shape[0]
    im = imresize(im, (desired_sz[0], int(np.round(target_ds * im.shape[1]))))
    d = (im.shape[1] - desired_sz[1]) / 2
    im = im[:, d:d+desired_sz[1]]
    return im

split = 'test'
frames_per_ex = 10
max_clip = 100 # a limit to the batch file size to run on my container

# calculate num_im,skip, parts, step_im
s_num_frames = int(round(s_hz * s_length))
s_orig_ms_per_frame = 1000/s_hz
s_targ_ms_per_frame = 1000.0/frate
skip = int(s_targ_ms_per_frame/s_orig_ms_per_frame)
s_actual_hz = s_hz/skip
s_actual_ms_per_frame = 1000/s_actual_hz
s_total_clips = int(s_num_frames/(frames_per_ex*skip))
parts = int(s_total_clips/max_clip) + 1
step_im = max_clip * frames_per_ex
suffix = '_P'+str(parts)+'_'
num_im = s_total_clips*frames_per_ex 

X = np.zeros((num_im,) + desired_im_sz + (3,), np.uint8)
source_list = [fname_in]*num_im
ct = 0
for i,im_out in enumerate(vid):
    if (i % skip == 0):
        im = vid.get_data(i)
        #print i
        X[ct] = process_im(im, desired_im_sz)
        ct = ct + 1
        if (ct == num_im):
            break

if not (os.path.isdir(home+"/test_avi")):
    os.mkdir(home+'/test_avi')
else:
    allfiles =os.listdir(home+'/test_avi')
    for temp in allfiles:
        os.remove(home+'/test_avi'+'/'+temp)
    os.rmdir(home+'/test_avi')
    os.mkdir(home+'/test_avi')
    
for part in range(1,parts+1):
    xbeg = int((part-1)*step_im)
    if (part == parts):
        xend = num_im    
    else:
        xend = int(part*step_im) 
    print part,xbeg,xend
    hkl.dump(X[xbeg:xend], os.path.join(home,'test_avi', 'X_' + split + suffix+ str(part)+'.hkl'))
    hkl.dump(source_list[xbeg:xend], os.path.join(home,'test_avi', 'sources_' + split + suffix + str(part)+ '.hkl'))


1 0 40


In [27]:
'''
Evaluate trained PredNet on test video.
Calculates mean-squared error

SS 12/5/2016:
modified to read in batched .hkl files

'''

import os
import numpy as np
import pandas as pd
from six.moves import cPickle
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from keras import backend as K
from keras.models import Model, model_from_json
from keras.layers import Input, Dense, Flatten

from prednet import PredNet
from data_utils import SequenceGenerator
#from kitti_settings_ss import *
DATA_DIR = './test_avi/'
WEIGHTS_DIR = './model_data/'
RESULTS_SAVE_DIR = './test_results_avi/'

# these are set in the cell above, uncomment this and this cell should
# work for avi's less than 1000 frames (1000 clips at 10 frames/clip)
# This would be 500 sec at 2Hz or 200 sec at 5Hz 
#parts = 1
#suffix = '_P1_'

n_plot = 40
batch_size = 10
nt = 10

weights_file = os.path.join(WEIGHTS_DIR, 'prednet_kitti_weights.hdf5')
json_file = os.path.join(WEIGHTS_DIR, 'prednet_kitti_model.json')

# Load trained model
f = open(json_file, 'r')
json_string = f.read()
f.close()
train_model = model_from_json(json_string, custom_objects = {'PredNet': PredNet})
train_model.load_weights(weights_file)

# Create testing model (to output predictions)
layer_config = train_model.layers[1].get_config()
layer_config['output_mode'] = 'prediction'
dim_ordering = layer_config['dim_ordering']
test_prednet = PredNet(weights=train_model.layers[1].get_weights(), **layer_config)
input_shape = list(train_model.layers[0].batch_input_shape[1:])
input_shape[0] = nt
inputs = Input(shape=tuple(input_shape))
predictions = test_prednet(inputs)
test_model = Model(input=inputs, output=predictions)
if not os.path.exists(RESULTS_SAVE_DIR): os.mkdir(RESULTS_SAVE_DIR)
f = open(RESULTS_SAVE_DIR + 'prediction_scores.txt', 'w')

for part in range(1,parts+1):

    curr_test = 'X_test'+suffix+str(part)+'.hkl'
    curr_sources = 'sources_test'+suffix+str(part)+'.hkl'
    test_file = os.path.join(DATA_DIR, curr_test)
    test_sources = os.path.join(DATA_DIR, curr_sources)

    test_generator = SequenceGenerator(test_file, test_sources, nt, sequence_start_mode='unique', dim_ordering=dim_ordering)
    X_test = test_generator.create_all()
    X_hat = test_model.predict(X_test, batch_size)
    if dim_ordering == 'th':
        X_test = np.transpose(X_test, (0, 1, 3, 4, 2))
        X_hat = np.transpose(X_hat, (0, 1, 3, 4, 2))

    curr_mse_frame2 = 'mse_frame2'+suffix+str(part)+'.csv'
    mse_frame2_out = os.path.join(RESULTS_SAVE_DIR, curr_mse_frame2)
    
    mse_point = (X_test[:, 1:] - X_hat[:, 1:])**2
    mse_frame2 = np.squeeze(np.apply_over_axes(np.mean, mse_point, [2,3,4]))
    np.savetxt(mse_frame2_out, mse_frame2, delimiter=",")

    print 'mse_frame2'
    print type(mse_frame2)
    print mse_frame2.shape
    
    # Compare overall MSE's write results to prediction_scores.txt
    mse_model = np.mean( (X_test[:, 1:] - X_hat[:, 1:])**2 )  # look at all timesteps except the first
    mse_prev = np.mean( (X_test[:, :-1] - X_test[:, 1:])**2 )
    mse_last = np.mean( (X_test[:, -1] - X_hat[:, -1])**2 )  # look only the last frame
    f.write("part number: %d\n" % part)
    f.write("Model MSE: %f\n" % mse_model)
    f.write("Last Frame MSE: %f" % mse_last)
    f.write("Previous Frame MSE: %f" % mse_prev)
    
f.close()


mse_frame2
<type 'numpy.ndarray'>
(4, 9)


In [60]:
# see https://github.com/startupml/video/blob/master/results/README.md for rationale for these thresholds
if (frate == 2.0):
    threshold = 0.00777565 
else: #frate == 5.0, as determined by first cell
    threshold = 0.004869763 
    
mse_clip = pd.Series(np.squeeze(np.apply_over_axes(np.mean, mse_frame2, [1])))

clip_size = mse_clip.shape[0]
t_start = pd.Series(np.zeros(clip_size))
t_end = pd.Series(np.zeros(clip_size))
above_threshold = pd.Series(np.zeros(clip_size))

for i in range(0,clip_size):
    if (mse_clip[i] >= threshold):
        above_threshold[i] = 1
    t_start[i] = i*(frames_per_ex*skip*s_orig_ms_per_frame)
    t_end[i] = (i+1)*(frames_per_ex*skip*s_orig_ms_per_frame) - (skip*s_orig_ms_per_frame)

mse_clip_out = pd.concat([mse_clip,t_start,t_end,above_threshold],axis=1)
mse_clip_out.columns = ['MSE_ave_over_clip','time_start','time_end','above_threshold']    
mse_clip_out.to_csv(os.path.join(RESULTS_SAVE_DIR, 'test_avi_mse_clip.csv'))

print 'Assessing possible accidents'
if (np.sum(mse_clip_out.above_threshold) == 0):
    print "No possible accidents found"
else:
    for i in range(0,clip_size):
        if (mse_clip_out.above_threshold.iloc[i] == 1):
            print 'Possible accident in clip ',i, 'starting time = ', mse_clip_out.time_start.iloc[i], \
            'ending time = ',mse_clip_out.time_end.iloc[i]


Assessing possible accidents
No possible accidents found
